In [1]:
import numpy as np
import pandas as pd
import itertools
import re
import os
import difflib
import random
import requests
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import operator

import Pre_process.pre_process as pre_process 
import M_Arc_Dict.model_dict as model_dict
import M_Create.rl_model_search as rl_model_search

from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

C:\Users\netlab\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [2]:
#adding key, child, q value to dictonary or q table
def add_q(key,q_dict,q): #q is the q value
    if key in q_dict:
#         print('true')
        q_dict[key]= q
    else:
#         print('false')
        d_tmp = dict()
        q_dict[key]= q
    
    return q_dict

#get q value from dictonary or q table
def get_q(key,q_dict):
    if key in q_dict:
#         print('true')
        q = q_dict[key]
    
        return q
    
    else:
        return 'nan'
        
#store dictonary or q table
def q_store(file_name,q_table):
    pickle_out =  open(file_name+".pickle",'wb')
    pickle.dump(q_table,pickle_out)
    pickle_out.close()
    
    
#load stored dictonary or q table
def q_load(file_name):
    pickle_in = open(file_name+".pickle","rb")
    tmp_dict = pickle.load(pickle_in)
#     print(tmp_dict)
    return tmp_dict

def q_fun(alpha,old_value,reward,gamma,next_max_or_min):
    new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max_or_min)
    return new_value

In [3]:
#LSTM00 Hierarchical Q Tables structure
# lstm00 = ['darchi','drnn','dlstm00','dlstm00_01','dlstm00_02']

def add_qtable(key,q,table_name,alpha,gamma,next_max_or_min):
        table_save = table_save_folder+table_name
        new_q = 0
        
        if not os.path.isfile(table_save+'.pickle'):
            print('Create new table')
            dtmp= dict()
            q_dict = dtmp
            old_value = 0
            reward = q
            new_q = q_fun(alpha,old_value,reward,gamma,next_max_or_min)
#             print('new_q',new_q)
            add_q(key,q_dict,new_q)
            q_store(table_save,q_dict)
            
        else:
            dtmp = q_load(table_save)
            if key in dtmp:
                old_value = dtmp[key]
            else:
                old_value = 0
                
            reward = q
            new_q = q_fun(alpha,old_value,reward,gamma,next_max_or_min)
#             print('new_q',new_q)
            q_dict = dtmp
            add_q(key,q_dict,new_q)
            q_store(table_save,q_dict)
            
#             print(q_dict)

In [4]:
def get_max_keyvalue(next_table_name):
#     print('next table name',next_table_name)
    next_dtmp = q_load('q_tables/'+ next_table_name)
    max_key, max_value =max(next_dtmp.items(), key=operator.itemgetter(1))
    return max_key, max_value

In [5]:
def get_min_keyvalue(next_table_name):
#     print('next table name',next_table_name)
    next_dtmp = q_load('q_tables/'+ next_table_name)
    min_key, min_value =min(next_dtmp.items(), key=operator.itemgetter(1))
    return min_key, min_value

In [6]:
##update the hierichical Q-table
def htable_update(objective,model_info,q,alpha,gamma):
    model_name = model_info[1]
    for i in range(len(model_info)):
        
        if str(type(model_info[i])) == "<class 'list'>":
            for j in range(len(model_info[i])):
        
                if str(type(model_info[i][j])) == "<class 'list'>":
                    for k in range(len(model_info[i][j])-1):
#                         print(model_info[i][j][k])
                        key = model_info[i][j][k+1]
                        table_name = model_name+'_'+str(j+k)
                        next_table_name =  model_name+'_'+str(j+k+1)
                        
#                         print('current third',table_name,'next',next_table_name)
                        
                        if not os.path.isfile(next_table_name+'.pickle'):
                            if objective == 'max':
                                max_value = 0
                            else:
                                max_value = 0
                        else:
                            if objective == 'max':
                                max_key, max_value = get_max_keyvalue(next_table_name)
                            else:
                                max_key, max_value = get_min_keyvalue(next_table_name)
                        
                        add_qtable(key,q,table_name,alpha,gamma,max_value) 
                else:
                   
                    table_name = model_name+'_'+str(j)
                    next_table_name =  model_name+'_'+str(j+1)
                    
#                     print('current second',table_name,'next',next_table_name)
                    
                    if not os.path.isfile(next_table_name+'.pickle'):
                        if objective == 'max':
                            max_value = 0
                        else:
                            max_value = 0
                    else:
                        if objective == 'max':
                            max_key, max_value = get_max_keyvalue(next_table_name)
                        else:
                            max_key, max_value = get_min_keyvalue(next_table_name)
                    
                    if str(type(model_info[i][j+1])) == "<class 'list'>":
                        key = model_info[i][j+1][0]
                    else:
                        key = model_info[i][j+1]
                    add_qtable(key,q,table_name,alpha,gamma,max_value) 
        else:
            table_name = model_info[i]
#             print('current first tow',table_name)
            if str(type(model_info[i+1])) == "<class 'list'>":
                key = model_info[i+1][0]
                
                next_table_name =  model_name+'_'+str(i+1)
                if not os.path.isfile(next_table_name+'.pickle'):
                    if objective == 'max':
                        max_value = 0
                    else:
                        max_value = 0
                else:
                    if objective == 'max':
                        max_key, max_value = get_max_keyvalue(next_table_name)
                    else:
                        max_key, max_value = get_min_keyvalue(next_table_name)
                
            else:
                key = model_info[i+1]
                
                next_table_name = model_info[i+1]
                if not os.path.isfile(next_table_name+'.pickle'):
                    if objective == 'max':
                        max_value = 0
                    else:
                        max_value = 0
                else:
                    if objective == 'max':
                        max_key, max_value = get_max_keyvalue(next_table_name)
                    else:
                        max_key, max_value = get_min_keyvalue(next_table_name)


            add_qtable(key,q,table_name,alpha,gamma,max_value)    
#             add_qtable(key,q,table_name)

In [7]:
def exploid_1(objective,model_win):
    file_name = model_win[1]
    for i in range(len(model_win[2])):
        name = -1
        if str(type(model_win[2][i])) == "<class 'list'>":
            for j in range(len(model_win[2][i])): 
                key = model_win[2][i][j]
                name = file_name + '_' + str(i+j-1)
                if objective == 'max':
                    max_key, max_value = get_max_keyvalue(name)
                else:
                    max_key, max_value = get_min_keyvalue(name)
                if max_key != key:
                    model_win[2][i][j] = max_key
                    print('replace', key, 'with', max_key)
                    return [model_win, 'r']
                else:
#                     print('use the original', key,'and search for next config')
                    if i+j == len(model_win[2][i]):
                        return [model_win, 'nr']
    
        else:    
            if i == 0:
                name = file_name
            else:
                name = file_name +'_'+ str(i-1)
            if objective == 'max':
                max_key, max_value = get_max_keyvalue(name)
            else:
                max_key, max_value = get_min_keyvalue(name)
            key = model_win[2][i]
            if max_key != key:
                model_win[2][i] = max_key
                print('replace', key, 'with', max_key)
                return [model_win, 'r']
            
#             else:
#                 print('use the original', key,'and search for next config')


def exploid_2(model_win):
    model_win[0][2][0] = np.random.randint(0,21)
    return model_win

In [8]:
#Generating pairs for competition
import random
from itertools import zip_longest

def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # Taken from itertools recipes:
    # https://docs.python.org/2/library/itertools.html#recipes
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx
    args = [iter(iterable)] * n
    return zip_longest(fillvalue=fillvalue, *args)

def  get_pairs(data):

    random.shuffle(data)
    tmp_arr = []
    for first, second in grouper(data, 2):
        tmp_arr.append([first,second])
#         print (str(first), "and", str(second))
    return tmp_arr

In [9]:
#Competition Round
#get winner within pair
def get_winner(objective,first,second):
    eval_first = first[1]
    eval_second = second[1]
    if objective == 'max': #objective is to maximize
        if(eval_first > eval_second):
            winner = first[0]
        else:
            winner = second[0]
            
    else:#objective is to minimize   
        if(eval_first < eval_second):
            winner = first[0]
        else:
            winner = second[0]
                  
    return winner

In [10]:
def get_ini_model(archi_list, 
                  model_name_list,
                  min_num_layers,
                  max_num_layers,
                  min_num_cells,
                  max_num_cells,
                  layer_step_size,cell_step_size):
    #Construct P number of models to initialized the competition
    population = []
    for i in range(P):
        archi_type = np.random.choice(archi_list)
        model_name = np.random.choice(model_name_list)
        model_tmp = get_rand_rnn(min_num_layers,max_num_layers,min_num_cells,max_num_cells,layer_step_size,cell_step_size)
        population.append([archi_type,model_name,model_tmp])
    return population

In [11]:
import re
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution1D, Lambda, \
    Convolution2D,Convolution3D, Flatten, \
    Reshape, LSTM, Dropout, TimeDistributed, BatchNormalization,AveragePooling2D
from keras.regularizers import l2
from keras.optimizers import Adam

from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import time

class TimeHistory(keras.callbacks.Callback):

    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)
        

    
        
def generate_LSTM_model(input_shapes, output_shape,optimizer,rnn_layer_config,rnn_cell_config,dense_neuron_config,output_act, loss_fun,dropout, recurrent_dropout):

    model = Sequential()
    for i in range(rnn_layer_config):
        model.add(LSTM(units=rnn_cell_config, dropout=dropout, recurrent_dropout=recurrent_dropout,return_sequences=True, input_shape=input_shapes))    
    model.add(Flatten())
    for dense in dense_neuron_config:
        model.add(Dense(dense, activation=output_act))
    model.add(Dropout(dropout))
    model.add(Dense(output_shape))
    model.compile(loss=loss_fun, optimizer=optimizer)

    return model

def get_bi_LSTM_model(input_shapes, output_shape,optmizer,rnn_layer_config,rnn_cell_config,dense_neuron_config,output_act, loss_fun, dropout, recurrent_dropout):
    model = Sequential()
    for i in range(rnn_layer_config):
        if (i+1) == rnn_layer_config:
            model.add(Bidirectional(LSTM(units=rnn_cell_config, activation=output_act,dropout=dropout, recurrent_dropout=recurrent_dropout, return_sequences=True),merge_mode ='concat'))
        else:
            model.add(Bidirectional(LSTM(units=rnn_cell_config, activation=output_act,dropout=dropout, recurrent_dropout=recurrent_dropout, return_sequences=True), input_shape=input_shapes))    
   
    model.add(Flatten())
    
    for dense in dense_neuron_config:
        model.add(Dense(dense, activation=output_act))
        model.add(Dropout(dropout))
        
    model.add(Dense(output_shape))
    model.compile(optimizer=optimizer, loss=loss_fun)
    
    return model

In [12]:
# without time distributed
def generate_CNN_model(input_shape, output_shape, type_1_arr, type_2_arr, output_act, loss_tmp,
                       metrics=['loss']):
    filters = type_1_arr
    dense_layers = type_2_arr

    model = Sequential()

    if len(filters) == 1:
        model.add(Conv1D(filters=filters[0], kernel_size=3, padding='same', activation='relu', input_shape=input_shape))
    else:
        for fil_num in range(len(filters)):
            fil = filters[fil_num]
            if fil_num == 0:
                model.add(Conv1D(filters=filters[0], kernel_size=3, padding='same', activation='relu',
                                 input_shape=input_shape))
            else:
                model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())

    for dense in dense_layers:
        model.add(Dense(dense, activation=output_act))
    model.add(Dropout(0.5))
    model.add(Dense(output_shape, activation='relu'))
    model.compile(loss=loss_tmp, optimizer='adam', metrics=metrics)
    return model

In [13]:
def get_rand_rnn(min_num_layers,max_num_layers,min_num_cells,max_num_cells,layer_step_size,cell_step_size):
    total_layer = np.arange(min_num_layers, max_num_layers + 1, layer_step_size) #Generated total number of layers
    total_cells = np.arange(min_num_cells, max_num_cells + 1, cell_step_size) #Generated total number of cells or neurons

    #RNN block configuration
    rnn_layer_config = np.random.choice(total_layer) #Randomly generate rnn layers configuration
    rnn_cell_config = np.random.choice(total_cells) #Randomly generate rnn cells configuration

    #Dense block configuration
    dense_layer_config = np.random.choice(total_layer) #Randomly generate dense layers configuration
    dense_neuron_config = []
    
    for layer in range(dense_layer_config):
        tmp = np.random.choice(total_cells) #Randomly generate dense neurons configuration
        dense_neuron_config.append(tmp) #Adding each layer information realted to neurons
    return [rnn_layer_config, rnn_cell_config, dense_layer_config,dense_neuron_config]    


def construct_model(input_shapes,output_shape,optimizer,output_act,loss_fun,dropout,recurrent_dropout,model_info):
    model_type =model_info[1]
    if model_type ==  'mlstm00':
        rnn_layer_config = model_info[2][0]
        rnn_cell_config = model_info[2][1]
        dense_neuron_config = model_info[2][3]
        model = generate_LSTM_model(input_shapes, 
                                output_shape,
                                optimizer,
                                rnn_layer_config,
                                rnn_cell_config,
                                dense_neuron_config,
                                output_act, 
                                loss_fun,dropout, 
                                recurrent_dropout)
        
    elif model_type == 'mbilstm00':
        rnn_layer_config = model_info[2][0]
        rnn_cell_config = model_info[2][1]
        dense_neuron_config = model_info[2][3]   
        model = get_bi_LSTM_model(input_shapes, 
                                  output_shape,
                                  optimizer,
                                  rnn_layer_config,
                                  rnn_cell_config,
                                  dense_neuron_config,
                                  output_act, 
                                  loss_fun, 
                                  dropout, 
                                  recurrent_dropout)
        
    return model

In [14]:
def competition(objective, win_list, population):
    while len(win_list)>1: 
        #get pairs
        pop_index_arr = win_list
        compete_pairs =get_pairs(pop_index_arr)
        #Competition to replace models
        for i in compete_pairs:
            if any(elem is None for elem in i) == False:
                first = [i[0],population[i[0]][3]]
                second =[i[1],population[i[1]][3]]
                winner = get_winner(objective,first,second) #choose the winner
                model_win = population[winner][0:3]
#         win_list.append(winner) #store winner in the winner list for the next competition
    
                for j in i:
                    if j != winner: #findout the loser to replace
                        lose = j
                        win_list.remove(lose)
                
#                         print(lose)
            
                if np.random.uniform(0,1)< threshold[0]:
                    tmp_model = exploid_1(objective, model_win)
                    if tmp_model[1] == 'nr':
                        tmp_model = exploid_2(tmp_model)[0]
                    else:#2
                        tmp_model = tmp_model[0]
                    
                        print('before 2',population[lose]) 
                        print(tmp_model)
                        print(population[lose][2])
                        population[lose] = tmp_model
                        print('after 2',population[lose])
                        

            
                else:#1
                    tmp_model = get_rand_rnn(min_num_layers,max_num_layers,min_num_cells,max_num_cells,layer_step_size,cell_step_size)  #Explore
                       
 
                    print('before 1',population[lose])       
                    population[lose][2] = tmp_model
                    print('after 1',population[lose])
        for i in population:
            print("updated pop", i)
    return population
        

In [15]:
# encoading scheme to construct Hierarchical Q Tables
###############
# "darchi" stores neural network architectures key, child with q values 
###############
# "drnn" stores RNN network architectures related key, child with q values
# "dcnn" stores CNN network architectures related key, child with q values
# "dmlp" stores MLP network architectures related key, child with q values
##############
#dlstm00
#dgru00
#dbilstm00
##############
# dlstm00_01
# dlstm00_02
# dlstm00_03
# dlstm00_04
#################






# output_act ='relu'
# loss_fun = 'mse'
# metrics = 'loss'
# optimizer = 'adam'
# dropout = 0.25
# recurrent_dropout=0.1


# nr_epochs = 200
# batch_size = 100
# verbose =2




# model_info = [archi_type,model_name,[5, 9, 3, [9, 3, 2]]]       

In [ ]:
##### choose hyperpameters
# lstm_layers = [128,128] 
# dense_layers = [64,32]
# dropout = 0.25
# recurrent_dropout = 0.1
# merge_mode = 'concat'

#choose the optimizer
# optimizer = 'adam'
# loss_fun = 'mse'

#Choose ahead and input window size for prediction
ahead = 1 # e.g., 1 day or 1 hours ahead
window_size = 3 # e.g, 3 days or 3 hours

# Choose the Input Features and Target 
features = ['cache_demand']
target = ['cache_demand']

# Define epochs and batch size for training
num_epochs = 500
num_batch = 1000

#Stattistic collector
log_loss = []
log_valloss = []

# load dataset
dataset = pd.read_csv('MVNO_daily_cache/MVNO_daily_cache_2000.csv')

scaler = MinMaxScaler(feature_range=(0, 1))# normalize features (0 ~ 1)

counter = 0
model = 0
for mvno in dataset['mvno'].unique():
    start = 0
    end = window_size
    num_samples = 0
    print("========",mvno,'=====')
    dataset_tmp =dataset[dataset['mvno']==mvno]
    dataset_tmp['cache_demand']=scaler.fit_transform(dataset_tmp[features])


    train_x = pd.DataFrame()
    train_y = pd.DataFrame()

    for i in range(len(dataset_tmp)-window_size):
        train_x = train_x.append(dataset_tmp.iloc[start:end],ignore_index = True)
        train_y = train_y.append(dataset_tmp.iloc[end],ignore_index = True)
        start = start+ahead
        end = end+ahead
        num_samples +=1

    # reshape the data    
    train_x = train_x[features].values.reshape(num_samples,window_size,len(features))
    train_y = train_y[target].values.reshape(num_samples,1)
        
    # Split the data into training and validation data   
    train_x =train_x[:int(len(train_x)*0.8)]
    train_y =train_y[:int(len(train_y)*0.8)]
    test_x =train_x[int(len(train_x)*0.8):]
    test_y =train_y[int(len(train_y)*0.8):]
    
    input_shapes = train_x.shape[1:3]
    output_shape = train_y.shape[1]

In [17]:
min_num_layers = 1 # Minimun number of layers a.k.a hidden layers
max_num_layers = 10 # Maximun number of layers a.k.a hidden layers

min_num_cells = 1 # Minimun number of cells or neurons or perceptrons
max_num_cells = 500 # Maximun number of cells or neurons or perceptrons

layer_step_size = 1
cell_step_size = 1


#initi
archi_table = 'marchi'
archi_list = ['mrnn']
model_name_list = ['mlstm00','mbilstm00']


P = 10
population = get_ini_model(archi_list, 
                  model_name_list,
                  min_num_layers,
                  max_num_layers,
                  min_num_cells,
                  max_num_cells,
                  layer_step_size,cell_step_size)
objective = 'min'
threshold = [0.5]
Round = 20

nr_epochs = 20
batch_size = 100
verbose =2


output_act ='relu'
loss_fun = 'mse'
metrics = 'loss'
optimizer = 'adam'
dropout = 0.25
recurrent_dropout=0.1


# archi_type = 'drnn'
# model_name = 'dlstm00'



table_save_folder = 'q_tables/'
alpha =0.1
gamma =0.5

for round in range(Round):
    print('Round------', round)
    #Training
    for i in population:
        print("updated pop check", i)
    
    for i in range(len(population)):
        model_info = population[i]
        print(model_info)
        model = construct_model(input_shapes,
                        output_shape,
                        optimizer,
                        output_act,
                        loss_fun,
                        dropout,
                        recurrent_dropout,
                        model_info)
        
        time_callback = model_dict.TimeHistory()
        history = model.fit(train_x, train_y,
                        epochs=nr_epochs, batch_size=batch_size,
                        validation_data=(test_x, test_y),
                        verbose=verbose,
                        callbacks=[time_callback])
        history_path ='logs/'
        file_name = history_path+model_info[1]
        
        obj = {str(model_info[2]):history.history}
        if not os.path.isfile(file_name+'.pickle'):
            q_store(file_name,obj)
        else:
            obj = q_load(file_name)
            obj[str(model_info[2])]=history.history
        q_store(file_name,obj)    
        
        file_name = history_path+'round00'
        obj = {round:{str(model_info[2]):history.history}}
        if not os.path.isfile(file_name+'.pickle'):
            q_store(file_name,obj)
        else:
            obj = q_load(file_name)
            obj[round]={str(model_info[2]):history.history}
        q_store(file_name,obj)   
        
        if len(population[i])!=4:
            population[i].append(np.mean(history.history['val_loss']))
        else:
            print('before loss', population[3])
            population[i][3] = np.mean(history.history['val_loss'])
            print('after loss', population[3])

    
    #Update q tables
    for model_info in population: 
        model_info_tmp = model_info[0:3]
        q=model_info[3]
        print('model_info',model_info)
        print("q", model_info[3])
        htable_update(objective,model_info_tmp,q,alpha,gamma)  

    #Competition##
    win_list = np.arange(P).tolist()
    pop_index_arr = win_list
    compete_pairs =get_pairs(pop_index_arr)#get pairs
    population = competition(objective,win_list,population)

Round------ 0
updated pop check ['mrnn', 'mbilstm00', [7, 357, 8, [336, 171, 126, 127, 372, 352, 447, 261]]]
updated pop check ['mrnn', 'mbilstm00', [10, 408, 6, [177, 81, 89, 387, 422, 25]]]
updated pop check ['mrnn', 'mlstm00', [7, 352, 10, [3, 279, 301, 398, 360, 111, 427, 335, 349, 446]]]
updated pop check ['mrnn', 'mbilstm00', [8, 303, 6, [274, 439, 133, 117, 146, 184]]]
updated pop check ['mrnn', 'mlstm00', [8, 143, 3, [414, 391, 43]]]
updated pop check ['mrnn', 'mbilstm00', [10, 305, 7, [241, 2, 91, 291, 352, 267, 72]]]
updated pop check ['mrnn', 'mbilstm00', [4, 281, 2, [240, 253]]]
updated pop check ['mrnn', 'mlstm00', [1, 241, 7, [192, 293, 350, 240, 374, 13, 425]]]
updated pop check ['mrnn', 'mlstm00', [4, 208, 3, [306, 300, 494]]]
updated pop check ['mrnn', 'mlstm00', [3, 484, 4, [429, 156, 165, 192]]]
['mrnn', 'mbilstm00', [7, 357, 8, [336, 171, 126, 127, 372, 352, 447, 261]]]
Train on 290 samples, validate on 58 samples
Epoch 1/20
 - 7s - loss: 0.0703 - val_loss: 0.0202
E

Epoch 9/20
 - 0s - loss: 0.0177 - val_loss: 0.0164
Epoch 10/20
 - 0s - loss: 0.0168 - val_loss: 0.0143
Epoch 11/20
 - 0s - loss: 0.0169 - val_loss: 0.0134
Epoch 12/20
 - 0s - loss: 0.0169 - val_loss: 0.0139
Epoch 13/20
 - 0s - loss: 0.0165 - val_loss: 0.0153
Epoch 14/20
 - 0s - loss: 0.0160 - val_loss: 0.0148
Epoch 15/20
 - 0s - loss: 0.0149 - val_loss: 0.0135
Epoch 16/20
 - 0s - loss: 0.0132 - val_loss: 0.0120
Epoch 17/20
 - 0s - loss: 0.0143 - val_loss: 0.0107
Epoch 18/20
 - 0s - loss: 0.0148 - val_loss: 0.0105
Epoch 19/20
 - 0s - loss: 0.0122 - val_loss: 0.0106
Epoch 20/20
 - 0s - loss: 0.0141 - val_loss: 0.0107
['mrnn', 'mlstm00', [1, 241, 7, [192, 293, 350, 240, 374, 13, 425]]]
Train on 290 samples, validate on 58 samples
Epoch 1/20
 - 1s - loss: 0.0706 - val_loss: 0.0230
Epoch 2/20
 - 0s - loss: 0.0325 - val_loss: 0.0261
Epoch 3/20
 - 0s - loss: 0.0223 - val_loss: 0.0127
Epoch 4/20
 - 0s - loss: 0.0160 - val_loss: 0.0106
Epoch 5/20
 - 0s - loss: 0.0185 - val_loss: 0.0114
Epoch 6/

TypeError: 'NoneType' object is not subscriptable

In [18]:
for round in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]:
    print('Round------', round)
    #Training
    for i in population:
        print("updated pop check", i)
    
    for i in range(len(population)):
        model_info = population[i]
        print(model_info)
        model = construct_model(input_shapes,
                        output_shape,
                        optimizer,
                        output_act,
                        loss_fun,
                        dropout,
                        recurrent_dropout,
                        model_info)
        
        time_callback = model_dict.TimeHistory()
        history = model.fit(train_x, train_y,
                        epochs=nr_epochs, batch_size=batch_size,
                        validation_data=(test_x, test_y),
                        verbose=verbose,
                        callbacks=[time_callback])
        history_path ='logs/'
        file_name = history_path+model_info[1]
        
        obj = {str(model_info[2]):history.history}
        if not os.path.isfile(file_name+'.pickle'):
            q_store(file_name,obj)
        else:
            obj = q_load(file_name)
            obj[str(model_info[2])]=history.history
        q_store(file_name,obj)    
        
        file_name = history_path+'round00'
        obj = {round:{str(model_info[2]):history.history}}
        if not os.path.isfile(file_name+'.pickle'):
            q_store(file_name,obj)
        else:
            obj = q_load(file_name)
            obj[round]={str(model_info[2]):history.history}
        q_store(file_name,obj)   
        
        if len(population[i])!=4:
            population[i].append(np.mean(history.history['val_loss']))
        else:
            print('before loss', population[3])
            population[i][3] = np.mean(history.history['val_loss'])
            print('after loss', population[3])

    
    #Update q tables
    for model_info in population: 
        model_info_tmp = model_info[0:3]
        q=model_info[3]
        print('model_info',model_info)
        print("q", model_info[3])
        htable_update(objective,model_info_tmp,q,alpha,gamma)  

    #Competition##
    win_list = np.arange(P).tolist()
    pop_index_arr = win_list
    compete_pairs =get_pairs(pop_index_arr)#get pairs
    population = competition(objective,win_list,population)

Round------ 1
updated pop check ['mrnn', 'mbilstm00', [7, 357, 8, [336, 171, 126, 127, 372, 352, 447, 261]], 0.016620833007618786]
updated pop check ['mrnn', 'mbilstm00', [10, 408, 6, [177, 81, 89, 387, 422, 25]], 0.01573546538129449]
updated pop check ['mrnn', 'mlstm00', [2, 9, 8, [294, 147, 294, 466, 412, 39, 81, 282]], 0.01713768388144672]
updated pop check ['mrnn', 'mbilstm00', [8, 303, 6, [274, 439, 133, 117, 146, 184]], 0.01590921194292605]
updated pop check ['mrnn', 'mlstm00', [8, 143, 3, [414, 391, 43]], 0.016242808382958173]
updated pop check ['mrnn', 'mbilstm00', [10, 305, 7, [241, 2, 91, 291, 352, 267, 72]], 0.015855315094813704]
updated pop check ['mrnn', 'mbilstm00', [4, 281, 2, [240, 253]], 0.01504777129739523]
updated pop check ['mrnn', 'mlstm00', [1, 241, 7, [192, 293, 350, 240, 374, 13, 425]], 0.013861258048564196]
updated pop check ['mrnn', 'mlstm00', [1, 484, 4, [429, 156, 165, 192]]]
updated pop check ['mrnn', 'mlstm00', [1, 484, 4, [429, 156, 165, 192]], 0.01474585

Epoch 6/20
 - 0s - loss: 0.0188 - val_loss: 0.0119
Epoch 7/20
 - 0s - loss: 0.0208 - val_loss: 0.0112
Epoch 8/20
 - 0s - loss: 0.0212 - val_loss: 0.0116
Epoch 9/20
 - 0s - loss: 0.0199 - val_loss: 0.0136
Epoch 10/20
 - 0s - loss: 0.0186 - val_loss: 0.0168
Epoch 11/20
 - 0s - loss: 0.0190 - val_loss: 0.0186
Epoch 12/20
 - 0s - loss: 0.0195 - val_loss: 0.0172
Epoch 13/20
 - 0s - loss: 0.0181 - val_loss: 0.0153
Epoch 14/20
 - 0s - loss: 0.0185 - val_loss: 0.0137
Epoch 15/20
 - 0s - loss: 0.0190 - val_loss: 0.0133
Epoch 16/20
 - 0s - loss: 0.0186 - val_loss: 0.0137
Epoch 17/20
 - 0s - loss: 0.0185 - val_loss: 0.0144
Epoch 18/20
 - 0s - loss: 0.0182 - val_loss: 0.0158
Epoch 19/20
 - 0s - loss: 0.0175 - val_loss: 0.0164
Epoch 20/20
 - 0s - loss: 0.0184 - val_loss: 0.0156
before loss ['mrnn', 'mbilstm00', [8, 303, 6, [274, 439, 133, 117, 146, 184]], 0.016067803371697664]
after loss ['mrnn', 'mbilstm00', [8, 303, 6, [274, 439, 133, 117, 146, 184]], 0.016067803371697664]
['mrnn', 'mbilstm00', [

Train on 290 samples, validate on 58 samples
Epoch 1/20
 - 5s - loss: 0.0693 - val_loss: 0.0197
Epoch 2/20
 - 0s - loss: 0.0296 - val_loss: 0.0328
Epoch 3/20
 - 0s - loss: 0.0251 - val_loss: 0.0163
Epoch 4/20
 - 0s - loss: 0.0176 - val_loss: 0.0116
Epoch 5/20
 - 0s - loss: 0.0191 - val_loss: 0.0118
Epoch 6/20
 - 0s - loss: 0.0176 - val_loss: 0.0151
Epoch 7/20
 - 0s - loss: 0.0174 - val_loss: 0.0201
Epoch 8/20
 - 0s - loss: 0.0175 - val_loss: 0.0179
Epoch 9/20
 - 0s - loss: 0.0172 - val_loss: 0.0144
Epoch 10/20
 - 0s - loss: 0.0177 - val_loss: 0.0135
Epoch 11/20
 - 0s - loss: 0.0179 - val_loss: 0.0142
Epoch 12/20
 - 0s - loss: 0.0170 - val_loss: 0.0157
Epoch 13/20
 - 0s - loss: 0.0175 - val_loss: 0.0164
Epoch 14/20
 - 0s - loss: 0.0172 - val_loss: 0.0160
Epoch 15/20
 - 0s - loss: 0.0174 - val_loss: 0.0148
Epoch 16/20
 - 0s - loss: 0.0171 - val_loss: 0.0145
Epoch 17/20
 - 0s - loss: 0.0176 - val_loss: 0.0152
Epoch 18/20
 - 0s - loss: 0.0171 - val_loss: 0.0160
Epoch 19/20
 - 0s - loss: 0.

Epoch 15/20
 - 0s - loss: 0.0192 - val_loss: 0.0142
Epoch 16/20
 - 0s - loss: 0.0191 - val_loss: 0.0132
Epoch 17/20
 - 0s - loss: 0.0185 - val_loss: 0.0136
Epoch 18/20
 - 0s - loss: 0.0198 - val_loss: 0.0152
Epoch 19/20
 - 0s - loss: 0.0180 - val_loss: 0.0157
Epoch 20/20
 - 0s - loss: 0.0194 - val_loss: 0.0158
['mrnn', 'mlstm00', [2, 32, 6, [379, 260, 13, 90, 471, 425]], 0.01403211518190801]
Train on 290 samples, validate on 58 samples
Epoch 1/20
 - 1s - loss: 0.0675 - val_loss: 0.0182
Epoch 2/20
 - 0s - loss: 0.0269 - val_loss: 0.0367
Epoch 3/20
 - 0s - loss: 0.0247 - val_loss: 0.0186
Epoch 4/20
 - 0s - loss: 0.0160 - val_loss: 0.0112
Epoch 5/20
 - 0s - loss: 0.0186 - val_loss: 0.0111
Epoch 6/20
 - 0s - loss: 0.0183 - val_loss: 0.0135
Epoch 7/20
 - 0s - loss: 0.0165 - val_loss: 0.0182
Epoch 8/20
 - 0s - loss: 0.0171 - val_loss: 0.0182
Epoch 9/20
 - 0s - loss: 0.0163 - val_loss: 0.0149
Epoch 10/20
 - 0s - loss: 0.0168 - val_loss: 0.0126
Epoch 11/20
 - 0s - loss: 0.0165 - val_loss: 0.01

Train on 290 samples, validate on 58 samples
Epoch 1/20
 - 9s - loss: 0.0669 - val_loss: 0.0161
Epoch 2/20
 - 0s - loss: 0.0252 - val_loss: 0.0429
Epoch 3/20
 - 0s - loss: 0.0260 - val_loss: 0.0167
Epoch 4/20
 - 0s - loss: 0.0175 - val_loss: 0.0115
Epoch 5/20
 - 0s - loss: 0.0194 - val_loss: 0.0117
Epoch 6/20
 - 0s - loss: 0.0188 - val_loss: 0.0142
Epoch 7/20
 - 0s - loss: 0.0170 - val_loss: 0.0188
Epoch 8/20
 - 0s - loss: 0.0176 - val_loss: 0.0193
Epoch 9/20
 - 0s - loss: 0.0172 - val_loss: 0.0161
Epoch 10/20
 - 0s - loss: 0.0170 - val_loss: 0.0145
Epoch 11/20
 - 0s - loss: 0.0170 - val_loss: 0.0143
Epoch 12/20
 - 0s - loss: 0.0172 - val_loss: 0.0144
Epoch 13/20
 - 1s - loss: 0.0172 - val_loss: 0.0161
Epoch 14/20
 - 0s - loss: 0.0174 - val_loss: 0.0169
Epoch 15/20
 - 0s - loss: 0.0171 - val_loss: 0.0161
Epoch 16/20
 - 0s - loss: 0.0171 - val_loss: 0.0149
Epoch 17/20
 - 1s - loss: 0.0169 - val_loss: 0.0150
Epoch 18/20
 - 0s - loss: 0.0174 - val_loss: 0.0150
Epoch 19/20
 - 0s - loss: 0.

...

Epoch 18/20
 - 0s - loss: 0.0179 - val_loss: 0.0140
Epoch 19/20
 - 1s - loss: 0.0185 - val_loss: 0.0138
Epoch 20/20
 - 1s - loss: 0.0184 - val_loss: 0.0146
['mrnn', 'mbilstm00', [10, 416, 5, [198, 401, 19, 368, 219]], 0.015763756167143584]
Train on 290 samples, validate on 58 samples
Epoch 1/20
 - 12s - loss: 0.0726 - val_loss: 0.0280
Epoch 2/20
 - 1s - loss: 0.0461 - val_loss: 0.0116
Epoch 3/20
 - 1s - loss: 0.0213 - val_loss: 0.0333
Epoch 4/20
 - 1s - loss: 0.0253 - val_loss: 0.0178
Epoch 5/20
 - 1s - loss: 0.0179 - val_loss: 0.0132
Epoch 6/20
 - 1s - loss: 0.0177 - val_loss: 0.0129
Epoch 7/20
 - 1s - loss: 0.0176 - val_loss: 0.0147
Epoch 8/20
 - 1s - loss: 0.0174 - val_loss: 0.0170
Epoch 9/20
 - 1s - loss: 0.0173 - val_loss: 0.0170
Epoch 10/20
 - 1s - loss: 0.0173 - val_loss: 0.0162
Epoch 11/20
 - 1s - loss: 0.0175 - val_loss: 0.0148
Epoch 12/20
 - 1s - loss: 0.0175 - val_loss: 0.0143
Epoch 13/20
 - 1s - loss: 0.0174 - val_loss: 0.0148
Epoch 14/20
 - 1s - loss: 0.0171 - val_loss: 0.

 - 1s - loss: 0.0189 - val_loss: 0.0120
Epoch 9/20
 - 1s - loss: 0.0180 - val_loss: 0.0131
Epoch 10/20
 - 1s - loss: 0.0174 - val_loss: 0.0154
Epoch 11/20
 - 1s - loss: 0.0173 - val_loss: 0.0175
Epoch 12/20
 - 1s - loss: 0.0178 - val_loss: 0.0181
Epoch 13/20
 - 1s - loss: 0.0174 - val_loss: 0.0174
Epoch 14/20
 - 0s - loss: 0.0170 - val_loss: 0.0159
Epoch 15/20
 - 1s - loss: 0.0168 - val_loss: 0.0147
Epoch 16/20
 - 1s - loss: 0.0174 - val_loss: 0.0143
Epoch 17/20
 - 0s - loss: 0.0173 - val_loss: 0.0144
Epoch 18/20
 - 1s - loss: 0.0169 - val_loss: 0.0148
Epoch 19/20
 - 1s - loss: 0.0167 - val_loss: 0.0156
Epoch 20/20
 - 1s - loss: 0.0166 - val_loss: 0.0161
['mrnn', 'mbilstm00', [9, 99, 8, [423, 108, 396, 131, 127, 333, 138, 316]]]
Train on 290 samples, validate on 58 samples
Epoch 1/20
 - 12s - loss: 0.0696 - val_loss: 0.0204
Epoch 2/20
 - 1s - loss: 0.0319 - val_loss: 0.0332
Epoch 3/20
 - 1s - loss: 0.0245 - val_loss: 0.0180
Epoch 4/20
 - 1s - loss: 0.0190 - val_loss: 0.0114
Epoch 5/20


Train on 290 samples, validate on 58 samples
Epoch 1/20
 - 6s - loss: 0.0710 - val_loss: 0.0228
Epoch 2/20
 - 0s - loss: 0.0333 - val_loss: 0.0362
Epoch 3/20
 - 0s - loss: 0.0279 - val_loss: 0.0152
Epoch 4/20
 - 0s - loss: 0.0195 - val_loss: 0.0112
Epoch 5/20
 - 0s - loss: 0.0202 - val_loss: 0.0117
Epoch 6/20
 - 0s - loss: 0.0181 - val_loss: 0.0152
Epoch 7/20
 - 0s - loss: 0.0178 - val_loss: 0.0199
Epoch 8/20
 - 0s - loss: 0.0183 - val_loss: 0.0181
Epoch 9/20
 - 0s - loss: 0.0178 - val_loss: 0.0150
Epoch 10/20
 - 0s - loss: 0.0179 - val_loss: 0.0140
Epoch 11/20
 - 0s - loss: 0.0174 - val_loss: 0.0143
Epoch 12/20
 - 0s - loss: 0.0178 - val_loss: 0.0151
Epoch 13/20
 - 0s - loss: 0.0172 - val_loss: 0.0160
Epoch 14/20
 - 0s - loss: 0.0172 - val_loss: 0.0166
Epoch 15/20
 - 0s - loss: 0.0175 - val_loss: 0.0163
Epoch 16/20
 - 0s - loss: 0.0170 - val_loss: 0.0158
Epoch 17/20
 - 0s - loss: 0.0178 - val_loss: 0.0147
Epoch 18/20
 - 0s - loss: 0.0172 - val_loss: 0.0150
Epoch 19/20
 - 0s - loss: 0.

Epoch 5/20
 - 0s - loss: 0.0222 - val_loss: 0.0113
Epoch 6/20
 - 0s - loss: 0.0205 - val_loss: 0.0136
Epoch 7/20
 - 0s - loss: 0.0193 - val_loss: 0.0178
Epoch 8/20
 - 0s - loss: 0.0210 - val_loss: 0.0166
Epoch 9/20
 - 0s - loss: 0.0195 - val_loss: 0.0138
Epoch 10/20
 - 0s - loss: 0.0199 - val_loss: 0.0126
Epoch 11/20
 - 0s - loss: 0.0197 - val_loss: 0.0130
Epoch 12/20
 - 0s - loss: 0.0195 - val_loss: 0.0143
Epoch 13/20
 - 0s - loss: 0.0181 - val_loss: 0.0159
Epoch 14/20
 - 0s - loss: 0.0194 - val_loss: 0.0151
Epoch 15/20
 - 0s - loss: 0.0188 - val_loss: 0.0141
Epoch 16/20
 - 0s - loss: 0.0188 - val_loss: 0.0136
Epoch 17/20
 - 0s - loss: 0.0186 - val_loss: 0.0141
Epoch 18/20
 - 0s - loss: 0.0181 - val_loss: 0.0150
Epoch 19/20
 - 0s - loss: 0.0189 - val_loss: 0.0150
Epoch 20/20
 - 0s - loss: 0.0171 - val_loss: 0.0142
['mrnn', 'mbilstm00', [8, 428, 9, [86, 246, 146, 275, 150, 35, 86, 329, 249]], 0.0171155025716871]
Train on 290 samples, validate on 58 samples
Epoch 1/20


ResourceExhaustedError:  OOM when allocating tensor with shape[856,1712] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node mul_43003 (defined at C:\Users\netlab\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3009) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_keras_scratch_graph_3745970]

Function call stack:
keras_scratch_graph


In [ ]:
competition(objective,win_list,population)

In [40]:
 for i in range(len(population)):
    print(population[i])

['mrnn', 'mlstm00', [6, 244, 7, [228, 73, 305, 7, 439, 468, 349]], 0.015826885607093572]
['mrnn', 'mbilstm00', [4, 416, 3, [40, 283, 148]], 0.011765528805553913]
['mrnn', 'mlstm00', [6, 412, 9, [489, 171, 92, 418, 302, 295, 36, 228, 141]], 0.01611568037420511]
['mrnn', 'mlstm00', [5, 379, 10, [141, 298]], 0.013764320388436318]
['mrnn', 'mlstm00', [7, 106, 6, [229, 215, 144, 181, 425, 211]], 0.01570639291778207]
['mrnn', 'mbilstm00', [4, 416, 3, [40, 283, 148]]]
['mrnn', 'mlstm00', [7, 59, 1, [274]], 0.013140555303543806]
['mrnn', 'mlstm00', [5, 379, 10, [141, 298]]]
['mrnn', 'mlstm00', [3, 4, 2, [193, 441]], 0.01596362644806504]
['mrnn', 'mbilstm00', [3, 149, 2, [489, 217]], 0.015961987860500813]
